In [1]:
# Imports
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os

import sys
sys.path.append('../../../')

from src import customFunc as cf
from src import ansatzs as anz
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.providers.fake_provider import FakeManila
import time

from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.providers.fake_provider import FakeGuadalupeV2
from qiskit.providers.aer import AerSimulator

## H8

In [16]:
# Cargamos los archivos

hamiltonians = {}
nuclear_repulsions = {}
ansatzes = {}

dist = 0.700

for n_atoms in [2, 4, 6, 8]:
    molecule_id = f"H{n_atoms}"

    with open(f"data/hamiltonian_{molecule_id}_{dist:.3f}.pkl", "rb") as f:
        hamiltonians[molecule_id] = pickle.load(f)

    with open(f"data/nuclear_repulsion_{molecule_id}_{dist:.3f}.pkl", "rb") as f:
        nuclear_repulsions[molecule_id] = pickle.load(f)

    with open(f"data/ansatz_{molecule_id}_{dist:.3f}.pkl", "rb") as f:
        ansatzes[molecule_id] = pickle.load(f)

In [ ]:
ansatz_naive = ansatzes['H8']
hamiltonan = hamiltonians['H8']

backend = FakePrague()

# Transpile circuit with a pass manager for a backend
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
t0 = time.time()
transpiled_ansatz_naive = pm.run(ansatz_naive)
t1 = time.time()
t_optimized=t1 - t0

# Show results
print(f"Transpilation time: {t_optimized} (s)")
print(f'Circuit info: {transpiled_ansatz_naive.count_ops()}')
print(f'Depth: {transpiled_ansatz_naive.depth()}')

# ansatz_naive.decompose().draw('mpl')

Transpilation time: 218.13428950309753 (s)
Circuit info: OrderedDict([('cx', 43715), ('rz', 28329), ('sx', 18722), ('x', 1825)])
Depth: 66048


In [19]:
ansatz_opt=anz.optimize_ansatz(ansatz_naive)

# Transpile circuit with a pass manager for a backend
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
t0 = time.time()
transpiled_ansatz_opt = pm.run(ansatz_opt)
t1 = time.time()
t_optimized=t1 - t0

# Show results
print(f"Transpilation time: {t_optimized} (s)")
print(f'Circuit info: {transpiled_ansatz_opt.count_ops()}')
print(f'Depth: {transpiled_ansatz_opt.depth()}')

Transpilation time: 134.2523169517517 (s)
Circuit info: OrderedDict([('cx', 44700), ('rz', 25178), ('sx', 16235), ('x', 1077)])
Depth: 63967


## O3

In [2]:
with open(f"data/ansatz_O3.pkl", "rb") as f:
    ansatz_naive = pickle.load(f)

backend = AerSimulator()

In [3]:
# Transpile circuit with a pass manager for a backend
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
t0 = time.time()
transpiled_ansatz_naive = pm.run(ansatz_naive)
t1 = time.time()
t_optimized=t1 - t0

# Show results
print(f"Transpilation time: {t_optimized} (s)")
print(f'Circuit info: {transpiled_ansatz_naive.count_ops()}')
print(f'Depth: {transpiled_ansatz_naive.depth()}')

Transpilation time: 2968.7533679008484 (s)
Circuit info: OrderedDict([('cx', 335184), ('u2', 35795), ('rz', 13680), ('u3', 22)])
Depth: 360453


In [ ]:
ansatz_opt=optimize_ansatz(ansatz_naive)

# Transpile circuit with a pass manager for a backend
pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
t0 = time.time()
transpiled_ansatz_opt = pm.run(ansatz_opt)
t1 = time.time()
t_optimized=t1 - t0

# Show results
print(f"Transpilation time: {t_optimized} (s)")
print(f'Circuit info: {transpiled_ansatz_opt.count_ops()}')
print(f'Depth: {transpiled_ansatz_opt.depth()}')

Transpilation time: 626.4570465087891 (s)
Circuit info: OrderedDict([('cx', 314780), ('u2', 20741), ('rz', 13680), ('unitary', 12026), ('u3', 22)])
Depth: 340121


In [6]:
ansatz_opt, transpiled_ansatz_opt, num_cx_prev=iterate_ansatz_opt(ansatz_naive, backend)

TypeError: UnitaryGate.__init__() missing 1 required positional argument: 'data'

In [ ]:
num_cx_prev

314780

In [5]:
from qiskit.transpiler import PassManager
from qiskit.transpiler.passes import (HighLevelSynthesis, InverseCancellation)
from qiskit.transpiler.passes.routing.commuting_2q_gate_routing import (SwapStrategy, FindCommutingPauliEvolutions, Commuting2qGateRouter)
from qiskit.circuit.library import CXGate, RZGate, RXGate, XGate, HGate, SXGate, SXdgGate, UnitaryGate
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
def optimize_ansatz(ansatz_naive):
    # Choose swap strategy (in this case -> line)
    num_qubits=ansatz_naive.num_qubits
    swap_strategy = SwapStrategy.from_line([i for i in range(num_qubits)])
    edge_coloring = {(idx, idx + 1): (idx + 1) % 2 for idx in range(num_qubits)}
    # Define pass manager
    init_cost_layer = PassManager([FindCommutingPauliEvolutions(), Commuting2qGateRouter(swap_strategy, edge_coloring,), HighLevelSynthesis(basis_gates=["x", "u", "h", "cx", "sx", "rz", "rx"]), InverseCancellation(gates_to_cancel=[CXGate(), XGate(), HGate(), UnitaryGate(),(RZGate(np.pi), RZGate(-np.pi)), (RZGate(np.pi/2), RZGate(-np.pi/2)), (SXGate(),SXdgGate())])])
    # Create a circuit for the 2 qubit gates and optimize it with the cost layer pass manager
    ansatz_opt=init_cost_layer.run(ansatz_naive)
    return ansatz_opt
def get_cx_count(ansatz, backend):
    pm = generate_preset_pass_manager(backend=backend, optimization_level=3)
    transpiled = pm.run(ansatz)
    ops = transpiled.count_ops()
    return transpiled, ops.get('cx', 0)
def iterate_ansatz_opt(ansatz_naive, backend):
    # Inicialization
    ansatz_opt = optimize_ansatz(ansatz_naive)
    transpiled_ansatz_opt, num_cx_prev = get_cx_count(ansatz_opt, backend)

    # Optimization loop
    while True:
        ansatz_opt_prev=ansatz_opt
        transpiled_ansatz_opt_prev=transpiled_ansatz_opt
        ansatz_opt = optimize_ansatz(ansatz_opt)
        transpiled_ansatz_opt, num_cx = get_cx_count(ansatz_opt, backend)
    
        if num_cx < num_cx_prev:
            num_cx_prev = num_cx
        else:
            break
    return ansatz_opt_prev, transpiled_ansatz_opt_prev, num_cx_prev